# Stochastic Variational Method with RL algorithms

In [1]:
import numpy as np
import gym
import svm_env as svm
import torch

## Expoloring environment

In [2]:
env = gym.make('svm_env:svmEnv-v0', file_sigmas ="./svmCodeSVD/sigmas.dat" )

obs_space = env.observation_space

print('###### Observation space ####### \n', obs_space)

state_size = env.observation_space.shape[-1]

print('###### Size of observation space ####### \n', state_size)

act_space = env.action_space

print('###### Action space ####### \n', act_space)

act_size = env.action_space.shape[-1]

print('###### Number of actions ####### \n', act_size)

state = env.reset()

print('##### State after reset ###### \n', state)

print('##### File where will be stored sigmas \n', env.file_sigmas)

act_space.sample()

###### Observation space ####### 
 Box(-inf, inf, (1,), float32)
###### Size of observation space ####### 
 1
###### Action space ####### 
 Box(-1.0, 1.0, (3,), float32)
###### Number of actions ####### 
 3
*****CALL RESET******
Action chosen at reset:  [0.]
Actions taken at reset:  []
Energies got at reset:  [0.0]
##### State after reset ###### 
 [0.]
##### File where will be stored sigmas 
 ./svmCodeSVD/sigmas.dat


array([ 0.6571757 ,  0.23591614, -0.36692822], dtype=float32)

## DDPG from `stable_baseline3`

In [ ]:
from stable_baselines3 import DDPG
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise

# The noise objects for DDPG
action_noise = NormalActionNoise(mean=np.zeros(act_size), sigma=0.1 * np.ones(act_size))

model = DDPG("MlpPolicy", env, action_noise = action_noise, \
            batch_size=128, gamma=1.0, verbose=1)

# (policy, env, learning_rate=0.001, buffer_size=1000000,learning_starts=100, batch_size=100, 
# tau=0.005, gamma=0.99, train_freq=(1, 'episode'),  gradient_steps=- 1, action_noise=None, 
# replay_buffer_class=None, replay_buffer_kwargs=None,  optimize_memory_usage=False, 
# tensorboard_log=None, create_eval_env=False, policy_kwargs=None,  verbose=0, seed=None, 
# device='auto', _init_setup_model=True)

model.learn(total_timesteps=1000, log_interval=5)

# learn(total_timesteps, callback=None, log_interval=4, eval_env=None, eval_freq=- 1,
# n_eval_episodes=5, tb_log_name='DDPG', eval_log_path=None, reset_num_timesteps=True)

## From my `ddpg_agent.py` code

In [3]:
from ddpg_agent import Agent

In [4]:
env = gym.make('svm_env:svmEnv-v0')
# Instance of the ddpg agent
agent = Agent(1, 3, random_seed=2)

### Training loop
def run_ddpg(max_t_step = 200, n_episodes = 3):        
    """Deep Deterministic Policy Gradient learning for Reacher Unity Environment.
    
    Params Input
    ==========
        n_episode (int): maximum number of episodes
        queue (int): number of consecutive episodes 
        
    Params Output
    ==========
        scores_all (list of floats): are the scores collected at the end of each episode
        
    """
    
    ##Inizialization
    scores_all = []                         
    for i_episode in range(1, n_episodes+1):
        state = env.reset()
        agent.reset()                  
        score = 0.0
        scores = []
        last_energies = []
        
        ## Training loop of each episode
        for t_step in range(max_t_step):
            action = agent.act(state)
            next_state, reward, done, info = env.step(action)                   
            agent.step(state, action, reward, next_state, done)
            score += reward
            state = next_state  
            if done:                                  
                break
        
        last_energies.append(state[0])
        scores.append(score)
        
        print('Episode {} ... Reward: {:.3f}'.format(i_episode, score))

    return scores

In [5]:
scores = run_ddpg()
torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')

*****CALL RESET******
Action chosen at reset:  [0.]
Actions taken at reset:  []
Energies got at reset:  [0.0]
****CALL STEP****
Action chosen at step:  [50.08913  39.656174 59.902096]
Triangular inequality not satisfied!
****CALL STEP****
Action chosen at step:  [ 26.44507 100.86172   0.     ]
**** TRIANGULAR INEQUALITY SATISFIED ****
With this action the energy is:  0.0
With this action the full dim is:  0  and princip dim is:  0
The new action:  [ 26.44507 100.86172   0.     ]  makes the energy positive:  True
The new action:  [ 26.44507 100.86172   0.     ]  makes the energy greater than:  0.0  the previous one:  True
The new action:  [ 26.44507 100.86172   0.     ]  makes the energy less than: -0.151 False
The new action:  [ 26.44507 100.86172   0.     ]  makes the energy nan:  False
The energy is greater than previous energy --> Set reward:  -1.0
Store the energy got!
****CALL STEP****
Action chosen at step:  [77.7017   44.600365 46.2004  ]
Triangular inequality not satisfied!
***

With this action the energy is:  -0.107714
With this action the full dim is:  6  and princip dim is:  6
The new action:  [42.484425    0.          0.79273224]  makes the energy positive:  False
The new action:  [42.484425    0.          0.79273224]  makes the energy greater than:  -0.107714  the previous one:  True
The new action:  [42.484425    0.          0.79273224]  makes the energy less than: -0.151 False
The new action:  [42.484425    0.          0.79273224]  makes the energy nan:  False
The energy is greater than previous energy --> Set reward:  -1.0
Store the energy got!
****CALL STEP****
Action chosen at step:  [46.249588 34.590763 22.928848]
Triangular inequality not satisfied!
****CALL STEP****
Action chosen at step:  [108.35698  80.90715   0.     ]
**** TRIANGULAR INEQUALITY SATISFIED ****
With this action the energy is:  -0.107714
With this action the full dim is:  6  and princip dim is:  6
The new action:  [108.35698  80.90715   0.     ]  makes the energy positive:  False

With this action the energy is:  -0.116555
With this action the full dim is:  9  and princip dim is:  9
The new action:  [ 87.287025 110.         0.      ]  makes the energy positive:  False
The new action:  [ 87.287025 110.         0.      ]  makes the energy greater than:  -0.116555  the previous one:  True
The new action:  [ 87.287025 110.         0.      ]  makes the energy less than: -0.151 False
The new action:  [ 87.287025 110.         0.      ]  makes the energy nan:  False
The energy is greater than previous energy --> Set reward:  -1.0
Store the energy got!
****CALL STEP****
Action chosen at step:  [50.092056 61.511192 58.355858]
Triangular inequality not satisfied!
****CALL STEP****
Action chosen at step:  [110.        105.456924    3.7639885]
**** TRIANGULAR INEQUALITY SATISFIED ****
With this action the energy is:  -0.116632
With this action the full dim is:  10  and princip dim is:  10
The new action:  [110.        105.456924    3.7639885]  makes the energy positive:  Fal

With this action the energy is:  -0.140175
With this action the full dim is:  12  and princip dim is:  12
The new action:  [13.880978 15.472881  0.      ]  makes the energy positive:  False
The new action:  [13.880978 15.472881  0.      ]  makes the energy greater than:  -0.140175  the previous one:  True
The new action:  [13.880978 15.472881  0.      ]  makes the energy less than: -0.151 False
The new action:  [13.880978 15.472881  0.      ]  makes the energy nan:  False
The energy is greater than previous energy --> Set reward:  -1.0
Store the energy got!
****CALL STEP****
Action chosen at step:  [17.86102  29.136763  0.      ]
**** TRIANGULAR INEQUALITY SATISFIED ****
With this action the energy is:  -0.140175
With this action the full dim is:  12  and princip dim is:  12
The new action:  [17.86102  29.136763  0.      ]  makes the energy positive:  False
The new action:  [17.86102  29.136763  0.      ]  makes the energy greater than:  -0.140175  the previous one:  True
The new actio

With this action the energy is:  -0.142964
With this action the full dim is:  16  and princip dim is:  16
The new action:  [47.73337   1.483818 55.293877]  makes the energy positive:  False
The new action:  [47.73337   1.483818 55.293877]  makes the energy greater than:  -0.142931  the previous one:  False
This action is NOT REMOVED from actions taken and sigmas, the energy is STORED!
Store the energy got!
Reward is positive! 16.00528
Calculate the diff between dim: 
Diff 2:  0
****CALL STEP****
Action chosen at step:  [47.997963 20.055508 45.572163]
Triangular inequality not satisfied!
****CALL STEP****
Action chosen at step:  [97.11958  88.801056  0.      ]
**** TRIANGULAR INEQUALITY SATISFIED ****
With this action the energy is:  -0.142964
With this action the full dim is:  16  and princip dim is:  16
The new action:  [97.11958  88.801056  0.      ]  makes the energy positive:  False
The new action:  [97.11958  88.801056  0.      ]  makes the energy greater than:  -0.142964  the pre

...

With this action the energy is:  -0.147554
With this action the full dim is:  12  and princip dim is:  12
The new action:  [ 0.        0.       22.263435]  makes the energy positive:  False
The new action:  [ 0.        0.       22.263435]  makes the energy greater than:  -0.147554  the previous one:  True
The new action:  [ 0.        0.       22.263435]  makes the energy less than: -0.151 False
The new action:  [ 0.        0.       22.263435]  makes the energy nan:  False
The energy is greater than previous energy --> Set reward:  -1.0
Store the energy got!
****CALL STEP****
Action chosen at step:  [ 0.        0.       55.272766]
**** TRIANGULAR INEQUALITY SATISFIED ****
With this action the energy is:  -0.147554
With this action the full dim is:  12  and princip dim is:  12
The new action:  [ 0.        0.       55.272766]  makes the energy positive:  False
The new action:  [ 0.        0.       55.272766]  makes the energy greater than:  -0.147554  the previous one:  True
The new actio

With this action the energy is:  -0.147554
With this action the full dim is:  12  and princip dim is:  12
The new action:  [ 0.        0.       19.734009]  makes the energy positive:  False
The new action:  [ 0.        0.       19.734009]  makes the energy greater than:  -0.147554  the previous one:  True
The new action:  [ 0.        0.       19.734009]  makes the energy less than: -0.151 False
The new action:  [ 0.        0.       19.734009]  makes the energy nan:  False
The energy is greater than previous energy --> Set reward:  -1.0
Store the energy got!
****CALL STEP****
Action chosen at step:  [6.863476 0.       0.      ]
**** TRIANGULAR INEQUALITY SATISFIED ****
With this action the energy is:  -0.147554
With this action the full dim is:  12  and princip dim is:  12
The new action:  [6.863476 0.       0.      ]  makes the energy positive:  False
The new action:  [6.863476 0.       0.      ]  makes the energy greater than:  -0.147554  the previous one:  True
The new action:  [6.86

With this action the energy is:  -0.147554
With this action the full dim is:  12  and princip dim is:  12
The new action:  [ 0.        0.       24.111668]  makes the energy positive:  False
The new action:  [ 0.        0.       24.111668]  makes the energy greater than:  -0.147554  the previous one:  True
The new action:  [ 0.        0.       24.111668]  makes the energy less than: -0.151 False
The new action:  [ 0.        0.       24.111668]  makes the energy nan:  False
The energy is greater than previous energy --> Set reward:  -1.0
Store the energy got!
****CALL STEP****
Action chosen at step:  [0.      0.      8.91011]
**** TRIANGULAR INEQUALITY SATISFIED ****
With this action the energy is:  -0.147554
With this action the full dim is:  12  and princip dim is:  12
The new action:  [0.      0.      8.91011]  makes the energy positive:  False
The new action:  [0.      0.      8.91011]  makes the energy greater than:  -0.147554  the previous one:  True
The new action:  [0.      0.   

With this action the energy is:  -0.147554
With this action the full dim is:  12  and princip dim is:  12
The new action:  [ 0.       0.      65.86098]  makes the energy positive:  False
The new action:  [ 0.       0.      65.86098]  makes the energy greater than:  -0.147554  the previous one:  True
The new action:  [ 0.       0.      65.86098]  makes the energy less than: -0.151 False
The new action:  [ 0.       0.      65.86098]  makes the energy nan:  False
The energy is greater than previous energy --> Set reward:  -1.0
Store the energy got!
****CALL STEP****
Action chosen at step:  [ 0.       0.      72.57361]
**** TRIANGULAR INEQUALITY SATISFIED ****
With this action the energy is:  -0.147554
With this action the full dim is:  12  and princip dim is:  12
The new action:  [ 0.       0.      72.57361]  makes the energy positive:  False
The new action:  [ 0.       0.      72.57361]  makes the energy greater than:  -0.147554  the previous one:  True
The new action:  [ 0.       0.    

With this action the energy is:  -0.147554
With this action the full dim is:  12  and princip dim is:  12
The new action:  [23.411383  0.        0.      ]  makes the energy positive:  False
The new action:  [23.411383  0.        0.      ]  makes the energy greater than:  -0.147554  the previous one:  True
The new action:  [23.411383  0.        0.      ]  makes the energy less than: -0.151 False
The new action:  [23.411383  0.        0.      ]  makes the energy nan:  False
The energy is greater than previous energy --> Set reward:  -1.0
Store the energy got!
****CALL STEP****
Action chosen at step:  [1.1912766 0.        0.       ]
**** TRIANGULAR INEQUALITY SATISFIED ****
With this action the energy is:  -0.147554
With this action the full dim is:  12  and princip dim is:  12
The new action:  [1.1912766 0.        0.       ]  makes the energy positive:  False
The new action:  [1.1912766 0.        0.       ]  makes the energy greater than:  -0.147554  the previous one:  True
The new actio

## Random search as in original SVM

In [ ]:
state = env.reset()
scores = []
step = 0
score = 0.0

while True:
    print(".....STEP.....", step)
    action = env.action_space.sample()
    next_state, reward, done, info = env.step(action)
    step = step + 1
    score += reward
    scores.append(score)
    state = next_state
    if done:
        break